<a href="https://colab.research.google.com/github/arraydevil/modelopredicao/blob/main/modelo_predicao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ciência de Dados na Saúde: Criando Modelos para Predição do Câncer

Estudos do Curso EBAC de Modelos para Predição de Câncer/Saúde.


## # Case: Classificação de Câncer por meio de microRNA

**Descrição dos Dados:** Os dados foram coletados do [The Cancer Genome Atlas]() (TCGA), que é um programa internacional e de referência mundial de caracterização de mais de 33 tipos de câncer. Os dados são reais e foram devidamente anonimizados. Cada linha representa a amostra retirada de uma pessoa. As colunas são os tipos de microRNA e cada entrada representa a intensidade com que aquele microRNA está expresso. Os valores de expressão variam entre $[0, \infty]$. Valores próximos a zero indicam pouca expressão enquanto que o contrário indica uma alta expressão. Os dados também apresentam rótulos (veja o atributo class) sendo TP (primary solid tumor) indicando tumor e NT (normal tissue).


**Objetivo:** Construir um modelo para predizer quando uma pessoa tem câncer dado um exame de sequenciamento do RNA.

## Leitura dos Dados

In [ ]:
import pandas as pd #manipulação de dados
import numpy as np #manipulação númerica
import seaborn as sns
import matplotlib.pyplot as plt #gráficos

from sklearn.model_selection import train_test_split #div de dados
from sklearn.linear_model import LogisticRegression #modelo de classificação
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA



In [ ]:
df = pd.read_csv("brca_mirnaseq(1).csv", sep=',',header=0, decimal=',')

In [ ]:
df.shape

## Análise Exploratória dos Dados

In [ ]:
ax = sns.countplot(x="class",data=df)

In [ ]:
df["class"].value_counts()

In [ ]:
df["class"].value_counts(normalize=True)

In [ ]:
df.describe()

## Estabelecendo um Baseline Comparativo

Antes de qualque modelagem vamos estabelecer um baseline, i.e., uma solução simples para o problema.

In [ ]:
x = df.drop("class", axis=1)
y = df["class"]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, stratify=y, random_state=42)

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

In [ ]:
lrc = LogisticRegression(random_state=42)

cv_list_lr_baseline = cross_val_score(
    lrc, X_train, y_train, cv=10, scoring="balanced_accuracy"
)

In [ ]:
mean_cv_baseline = np.mean(cv_list_lr_baseline)
std_cv_lr_baseline = np.std(cv_list_lr_baseline)

print(f"Performance (bac): {round(mean_cv_lr_baseline, 4)} +- {round(std_cv_lr_baseline, 4)}")

## Modelagem

In [ ]:
knn = Pipeline(
    {
        ('mms', MinMaxScaler()),
        ('skb', SelectKBest(chi2, k=10)),
        ('knn', KNeighborsClassifier(
            n_neighbors=5,
            p=2,
            weights='uniforme',
        ))
    }
)

cv_list_knn_euclid = cross_val_score(
    knn,
    X_train,
    y_train,
    cv=10,
    scoring="balanced_accuracy"
)

mean_cv_knn_euclid = np.mean(cv_list_knn_euclid)
std_cv_knn_euclid = np.std(cv_list_lr_baseline)

print(f"Performance (bac): {round(mean_cv_knn_euclid, 4)} +- {round(std_cv_knn_euclid, 4)}")

In [ ]:
lr = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(
            penalty="l1",
            C=1,
            fit_intercept=True,
            class_weight="balanced"
            solver= "liblinear",
            random_state=42
        )
        )
    ])

cv_list_lr_l1 = cross_val_score(
    lr,
    X_train,
    y_train,
    cv=10,
    scoring="balanced_accuracy"
)
mean_cv_lr_l1 = np.mean(cv_list_lr_l1)
std_cv_lr_l1 = np.std(cv_list_lr_l1)

print(f"Performance (bac): {round(mean_cv_lr_l1, 4)} +- {round(std_cv_lr_l1, 4)}")

In [ ]:
lr = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=10))
        ('lr', LogisticRegression(
            penalty="l2",
            C=1,
            fit_intercept=True,
            class_weight="balanced"
            solver= "liblinear",
            random_state=42
        )
        )
    ])

cv_list_lr_pca = cross_val_score(
    lr,
    X_train,
    y_train,
    cv=10,
    scoring="balanced_accuracy"
)
mean_cv_lr_pca = np.mean(cv_list_lr_pca)
std_cv_lr_pca = np.std(cv_list_lr_pca)


## Avaliação Experimental

In [ ]:
# resultados da cross-validacao
df_result_cv = pd.DataFrame(
    [cv_list_lr_baseline, cv_list_knn_euclid, cv_list_lr_l1, cv_list_lr_pca],
    index=["baseline", "kNN_euclid", "kNN_man" "LR-L2", "LR-L1", "LR-PCA"]
).T
df_result_cv


In [ ]:
df_res = df_result_cv.stack().to_frame("balanced_accuracy")
df_res.index.rename(["fold", "pipelines"], inplace=True)
df_res = df_res.reset_index()
df_res.head(12)

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.boxplot(x="pipelines",y="balanced_accuracy", data=df_res)
ax = sns.stripplot(x="pipelines", y="balanced_accuracy", data=df_res, color=".40")

In [ ]:
lr = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=10))
        ('lr', LogisticRegression(
            penalty="l2",
            C=1,
            fit_intercept=True,
            class_weight="balanced"
            solver= "liblinear",
            random_state=42
        )
        )
    ])
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
lr_pca_test = balanced_accuracy_score(y_test, y_pred)

print("Performance: ", round(lr_pca_test, 4))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_estimator(lr, X-test, y_test)
plt.show()

In [ ]:
ConfusionMatrixDisplay.from_estimator(lr, X_test, y_test, normalize='true')
plt.show()

## Referências & Links



1. [The Cancer Genome Atlas Program](https://www.cancer.gov/about-nci/organization/ccg/research/structural-genomics/tcga)
2. [Micro RNA](https://en.wikipedia.org/wiki/MicroRNA_sequencing)
3. [Sklearn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
